In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('WHO_Infrastructure_2010_2014.csv')
medical_doctors = pd.read_csv("Medical doctors (number).csv")
nursing_personnel = pd.read_csv("nursing personnel (number).csv")
pharmacist = pd.read_csv("Pharmacists  (per 10,000).csv")
community_HealthWorkers = pd.read_csv("Community Health Workers (number).csv")
health_ExpenditureUSD = pd.read_csv("Current health expenditure (CHE) per capita in US$.csv")
hospital_density = pd.read_csv("Hospital Density.csv")
percentage_health_exp_against_gdp = pd.read_csv('Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%).csv')
death_rate = pd.read_csv('crude_death_rate_data.csv')
life_expectancy = pd.read_csv('life_expectancy_1960-2022.csv')
population_data = pd.read_csv('population_1960-2022.csv')



In [3]:
medical_doctors.rename(columns={'FactValueNumeric': 'NO_medical_doc'}, inplace=True)

In [4]:
nursing_personnel.rename(columns={'FactValueNumeric': 'No_nurses'}, inplace=True)
nursing_personnel.head()

,ParentLocation,Location,Period,No_nurses
0,Americas,Brazil,2021,1181537
1,Africa,Mozambique,2021,10928
2,Europe,Estonia,2021,14854
3,Americas,Costa Rica,2021,15784
4,Americas,El Salvador,2021,16640


In [5]:
# Merge the 'medical_doctors' and 'nursing_personnel' DataFrames on 'Location' and 'Period' using an outer join
practitioners = medical_doctors.merge(nursing_personnel, on=['Location', 'Period'], how='outer')

# Drop unnecessary columns from the merged DataFrame
practitioners.drop(columns=['ParentLocation_y', 'Location type', 'Indicator'], axis=1, inplace=True)

# Rename the 'FactValueNumeric' column to 'No_doctors'
practitioners.rename(columns={'FactValueNumeric': 'No_doctors'}, inplace=True)

practitioners.head(3)

,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0
1,Africa,Chad,2021,1000.0,2677.0
2,Western Pacific,Tonga,2021,107.0,443.0


In [6]:
hospital_density.rename(columns={'Country': 'Location', 'Year': 'Period', 'Variable': 'Institution_name', 'Value':'Hosp_inst_no'}, inplace= True)

hospital_density.drop(columns=['VAR','UNIT','Measure','COU','YEA','Flag Codes','Flags'], axis=1, inplace=True)

hospital_density.head(3)

,Institution_name,Location,Period,Hosp_inst_no
0,Hospitals,Australia,2010,1345.0
1,Hospitals,Australia,2011,1345.0
2,Hospitals,Australia,2012,1347.0


In [7]:
# Merge the 'practitioners' DataFrame with the 'hospital_density' DataFrame on 'Location' and 'Period' using an outer join
infrastructure = practitioners.merge(hospital_density, on=['Location', 'Period'], how='outer')

# Merge the resulting 'infrastructure' DataFrame with the 'community_HealthWorkers' DataFrame on 'Location' and 'Period' using an outer join
infrastructure = infrastructure.merge(community_HealthWorkers, on=['Location', 'Period'], how='outer')

# Merge the updated 'infrastructure' DataFrame with the 'percentage_health_exp_against_gdp' DataFrame on 'Location' and 'Period' using an outer join
infrastructure = infrastructure.merge(percentage_health_exp_against_gdp, on=['Location', 'Period'], how='outer')

infrastructure.head(500)


,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,Comm_health_workerNO,(%) of health exp against GDP
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0,NaN,NaN,NaN,NaN
1,Africa,Chad,2021,1000.0,2677.0,NaN,NaN,8077.0,NaN
2,Western Pacific,Tonga,2021,107.0,443.0,NaN,NaN,NaN,NaN
3,Africa,Kenya,2021,11980.0,NaN,NaN,NaN,86490.0,NaN
4,South-East Asia,Bangladesh,2021,113477.0,77091.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
495,Europe,Iceland,2019,1404.0,5539.0,For-profit privately owned hospitals,0.0,NaN,8.53
496,Africa,Senegal,2019,1435.0,6151.0,NaN,NaN,1116.0,4.50
497,Africa,Rwanda,2019,1492.0,10409.0,NaN,NaN,NaN,6.33
498,Eastern Mediterranean,Lebanon,2019,15131.0,NaN,NaN,NaN,NaN,7.90


In [8]:
# Use the melt function to convert the data to long format

print(death_rate.head(3))
death_rate = pd.melt(death_rate, id_vars=['Country Name'], var_name='PERIOD', value_name='RATE')

death_rate.head(3)

  Country Name    1990    2000   2013   2014   2016  2016.1   2017   2018  \
0  Afghanistan  18.449  12.096  7.478  7.395  7.331   7.077  7.027  6.981   
1      Albania   5.526   5.798  7.819  7.868  7.947   8.035  8.150  8.308   
2      Algeria   5.731   5.037  4.673  4.555  4.437   4.472  4.542  4.482   

    2019    2020    2021  2022  
0  6.791   7.113   7.344   NaN  
1  8.480  10.785  11.325   NaN  
2  4.392   5.398   4.546   NaN  


,Country Name,PERIOD,RATE
0,Afghanistan,1990,18.449
1,Albania,1990,5.526
2,Algeria,1990,5.731


In [9]:
death_rate.rename(columns={'Country Name': 'Location', 'PERIOD':'Period', 'RATE':'Crude_death_rate' }, inplace= True)


In [10]:
# Convert the 'Period' column to numeric, coercing errors to NaN where necessary
death_rate['Period'] = pd.to_numeric(death_rate['Period'], errors='coerce')

# Cast the 'Period' column to integer type to ensure it contains only whole numbers
death_rate['Period'] = death_rate['Period'].astype('int')
death_rate.head()


,Location,Period,Crude_death_rate
0,Afghanistan,1990,18.449
1,Albania,1990,5.526
2,Algeria,1990,5.731
3,American Samoa,1990,NaN
4,Andorra,1990,NaN


In [11]:
# Merge the 'infrastructure' DataFrame with the 'death_rate' DataFrame on 'Location' and 'Period' using an outer join
infrastructure = infrastructure.merge(death_rate, on=['Location', 'Period'], how='outer')

infrastructure.head()




,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,Comm_health_workerNO,(%) of health exp against GDP,Crude_death_rate
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0,NaN,NaN,NaN,NaN,2.864
1,Africa,Chad,2021,1000.0,2677.0,NaN,NaN,8077.0,NaN,12.504
2,Western Pacific,Tonga,2021,107.0,443.0,NaN,NaN,NaN,NaN,6.970
3,Africa,Kenya,2021,11980.0,NaN,NaN,NaN,86490.0,NaN,8.056
4,South-East Asia,Bangladesh,2021,113477.0,77091.0,NaN,NaN,NaN,NaN,5.680


In [12]:
# Reshape the 'life_expectancy' DataFrame using the 'melt' function to pivot it from wide to long format
# The 'Country Name' column is used as the identifier variable, and the column names are transformed into the 'Period' column
# The values are transformed into the 'Expectancy_Age' column
life_expectancy = pd.melt(life_expectancy, id_vars=['Country Name'], var_name='Period', value_name='Expectancy_Age')

# Rename the 'Country Name' column to 'Location'
life_expectancy.rename(columns={'Country Name': 'Location'}, inplace=True)

# Display the first few rows of the reshaped 'life_expectancy' DataFrame
life_expectancy.head()


,Location,Period,Expectancy_Age
0,Aruba,1960,64.152000
1,Africa Eastern and Southern,1960,44.085552
2,Afghanistan,1960,32.535000
3,Africa Western and Central,1960,37.845152
4,Angola,1960,38.211000


In [13]:
life_expectancy['Period'] = pd.to_numeric(life_expectancy['Period'], errors='coerce')
life_expectancy['Period'] = life_expectancy['Period'].astype('int')

In [14]:
infrastructure = infrastructure.merge(life_expectancy, on = ['Location', 'Period'], how = 'outer')
infrastructure.head()




,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,Comm_health_workerNO,(%) of health exp against GDP,Crude_death_rate,Expectancy_Age
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0,NaN,NaN,NaN,NaN,2.864,76.936
1,Africa,Chad,2021,1000.0,2677.0,NaN,NaN,8077.0,NaN,12.504,52.525
2,Western Pacific,Tonga,2021,107.0,443.0,NaN,NaN,NaN,NaN,6.970,70.986
3,Africa,Kenya,2021,11980.0,NaN,NaN,NaN,86490.0,NaN,8.056,61.427
4,South-East Asia,Bangladesh,2021,113477.0,77091.0,NaN,NaN,NaN,NaN,5.680,72.381


In [15]:
# Reshape the 'population_data' DataFrame using the 'melt' function to pivot it from wide to long format
# The 'Country Name' column is used as the identifier variable, and the column names are transformed into the 'Period' column
# The values are transformed into the 'Population' column
population_data = pd.melt(population_data, id_vars=['Country Name'], var_name='Period', value_name='Population')

# Rename the 'Country Name' column to 'Location'
population_data.rename(columns={'Country Name': 'Location'}, inplace=True)

# Convert the 'Period' column to numeric, coercing errors to NaN where necessary
population_data['Period'] = pd.to_numeric(population_data['Period'], errors='coerce')

# Cast the 'Period' column to integer type to ensure it contains only whole numbers
population_data['Period'] = population_data['Period'].astype('int')

# Merge the 'infrastructure' DataFrame with the 'population_data' DataFrame on 'Location' and 'Period' using an outer join
infrastructure = infrastructure.merge(population_data, on=['Location', 'Period'], how='outer')

# Display the first few rows of the merged 'infrastructure' DataFrame
infrastructure.head()


,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,Comm_health_workerNO,(%) of health exp against GDP,Crude_death_rate,Expectancy_Age,Population
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0,NaN,NaN,NaN,NaN,2.864,76.936,35950396.0
1,Africa,Chad,2021,1000.0,2677.0,NaN,NaN,8077.0,NaN,12.504,52.525,17179740.0
2,Western Pacific,Tonga,2021,107.0,443.0,NaN,NaN,NaN,NaN,6.970,70.986,106017.0
3,Africa,Kenya,2021,11980.0,NaN,NaN,NaN,86490.0,NaN,8.056,61.427,53005614.0
4,South-East Asia,Bangladesh,2021,113477.0,77091.0,NaN,NaN,NaN,NaN,5.680,72.381,169356251.0


# `Data Cleaning`

In [16]:
def checker(dataframe):
    # Print the number of duplicated values in the dataframe
    print('Duplicated values:', dataframe.duplicated().sum())
    print('*******')
    # Print the percentage of null values for each column in the dataframe
    print('Null values:\n', dataframe.isna().sum() / len(dataframe) * 100)

In [17]:
infrastructure = infrastructure[infrastructure['Period']>= 2010]  # Filter the dataset for periods from 2010 and later

infrastructure.head()


,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,Comm_health_workerNO,(%) of health exp against GDP,Crude_death_rate,Expectancy_Age,Population
0,Eastern Mediterranean,Saudi Arabia,2021,100247.0,196795.0,NaN,NaN,NaN,NaN,2.864,76.936,35950396.0
1,Africa,Chad,2021,1000.0,2677.0,NaN,NaN,8077.0,NaN,12.504,52.525,17179740.0
2,Western Pacific,Tonga,2021,107.0,443.0,NaN,NaN,NaN,NaN,6.970,70.986,106017.0
3,Africa,Kenya,2021,11980.0,NaN,NaN,NaN,86490.0,NaN,8.056,61.427,53005614.0
4,South-East Asia,Bangladesh,2021,113477.0,77091.0,NaN,NaN,NaN,NaN,5.680,72.381,169356251.0


In [18]:
checker(infrastructure)

Duplicated values: 189
*******
Null values:
 ParentLocation_x                 45.333674
Location                          0.000000
Period                            0.000000
NO_medical_doc                   45.333674
No_nurses                        43.761186
Institution_name                 45.538226
Hosp_inst_no                     45.538226
Comm_health_workerNO             96.113526
(%) of health exp against GDP    32.638711
Crude_death_rate                 40.245462
Expectancy_Age                   14.190744
Population                        8.693429
dtype: float64


`Dealing with duplicated values`

In [19]:
infrastructure = infrastructure.drop_duplicates()
checker(infrastructure)


Duplicated values: 0
*******
Null values:
 ParentLocation_x                 45.919036
Location                          0.000000
Period                            0.000000
NO_medical_doc                   45.919036
No_nurses                        44.451723
Institution_name                 46.521682
Hosp_inst_no                     46.521682
Comm_health_workerNO             96.056596
(%) of health exp against GDP    33.093148
Crude_death_rate                 40.403511
Expectancy_Age                   14.162190
Population                        8.581161
dtype: float64


`Dealing with rows that have multiple null values`

In [20]:
# Define the subset of columns to consider
subset_columns = ['NO_medical_doc', 'No_nurses', 'Hosp_inst_no']

# Drop rows with null values in all specified columns, not either or, but all three
infrastructure = infrastructure.dropna(subset=subset_columns, how='all')

# Call the 'checker' function to perform further checks or actions on the 'infrastructure' DataFrame
checker(infrastructure)




Duplicated values: 0
*******
Null values:
 ParentLocation_x                 21.951220
Location                          0.000000
Period                            0.000000
NO_medical_doc                   21.951220
No_nurses                        19.833617
Institution_name                 22.820949
Hosp_inst_no                     22.820949
Comm_health_workerNO             94.668179
(%) of health exp against GDP    16.581584
Crude_death_rate                 38.910947
Expectancy_Age                   11.495557
Population                        9.775005
dtype: float64


Dropping Rows with All Null Values: Using the `dropna` method with the 'how' parameter set to 'all,' it removes rows in the DataFrame where all the values in the specified columns are null. This means that if any of these three columns contain at least one complete value, the row will be retained.

In [21]:
# Fill null values in the 'No_nurses' column with the mean value for each 'Location'
infrastructure['No_nurses'] = infrastructure.groupby(['Location'])['No_nurses'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'NO_medical_doc' column with the mean value for each 'Location'
infrastructure['NO_medical_doc'] = infrastructure.groupby(['Location'])['NO_medical_doc'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Hosp_inst_no' column with the mean value for each 'Location'
infrastructure['Hosp_inst_no'] = infrastructure.groupby(['Location'])['Hosp_inst_no'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the '(%) of health exp against GDP' column with the mean value for each 'Location'
infrastructure['(%) of health exp against GDP'] = infrastructure.groupby(['Location'])['(%) of health exp against GDP'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Crude_death_rate' column with the mean value for each 'Location'
infrastructure['Crude_death_rate'] = infrastructure.groupby(['Location'])['Crude_death_rate'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Expectancy_Age' column with the mean value for each 'Location'
infrastructure['Expectancy_Age'] = infrastructure.groupby(['Location'])['Expectancy_Age'].transform(lambda x: x.fillna(x.mean()))

# Drop the 'Comm_health_workerNO' column from the DataFrame
infrastructure.drop(columns='Comm_health_workerNO', axis=1, inplace=True)


checker(infrastructure)



Duplicated values: 0
*******
Null values:
 ParentLocation_x                 21.951220
Location                          0.000000
Period                            0.000000
NO_medical_doc                    9.321233
No_nurses                         9.302326
Institution_name                 22.820949
Hosp_inst_no                     22.726413
(%) of health exp against GDP     9.510304
Crude_death_rate                  9.756098
Expectancy_Age                   10.020798
Population                        9.775005
dtype: float64


In [22]:
# Replace 'Türkiye' with 'Turkey' in the 'Location' column
infrastructure['Location'] = infrastructure['Location'].replace('Türkiye', 'Turkey')

# Replace 'United States of America' with 'America' in the 'Location' column
infrastructure['Location'] = infrastructure['Location'].replace('United States of America', 'America')

# Replace 'Viet Nam' with 'Vietnam' in the 'Location' column
infrastructure['Location'] = infrastructure['Location'].replace('Viet Nam', 'Vietnam')

# Fill null values in the 'No_nurses' column with the mean value for each 'Location'
infrastructure['No_nurses'] = infrastructure.groupby(['Location'])['No_nurses'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'NO_medical_doc' column with the mean value for each 'Location'
infrastructure['NO_medical_doc'] = infrastructure.groupby(['Location'])['NO_medical_doc'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Hosp_inst_no' column with the mean value for each 'Location'
infrastructure['Hosp_inst_no'] = infrastructure.groupby(['Location'])['Hosp_inst_no'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the '(%) of health exp against GDP' column with the mean value for each 'Location'
infrastructure['(%) of health exp against GDP'] = infrastructure.groupby(['Location'])['(%) of health exp against GDP'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Crude_death_rate' column with the mean value for each 'Location'
infrastructure['Crude_death_rate'] = infrastructure.groupby(['Location'])['Crude_death_rate'].transform(lambda x: x.fillna(x.mean()))

# Fill null values in the 'Expectancy_Age' column with the mean value for each 'Location'
infrastructure['Expectancy_Age'] = infrastructure.groupby(['Location'])['Expectancy_Age'].transform(lambda x: x.fillna(x.mean()))

# Call the 'checker' function to perform further checks or actions on the 'infrastructure' DataFrame
checker(infrastructure)



Duplicated values: 0
*******
Null values:
 ParentLocation_x                 21.951220
Location                          0.000000
Period                            0.000000
NO_medical_doc                    9.321233
No_nurses                         9.302326
Institution_name                 22.820949
Hosp_inst_no                     22.726413
(%) of health exp against GDP     9.510304
Crude_death_rate                  9.756098
Expectancy_Age                   10.020798
Population                        9.775005
dtype: float64


In [23]:
# Define the subset of columns to consider
subset_columns = ['Crude_death_rate', 'Expectancy_Age', 'Population']

# Drop rows with null values in all specified columns, not either or, but all three
infrastructure = infrastructure.dropna(subset=subset_columns, how='all')

# Call the 'checker' function to perform further checks or actions on the 'infrastructure' DataFrame
checker(infrastructure)

Duplicated values: 0
*******
Null values:
 ParentLocation_x                 18.292172
Location                          0.000000
Period                            0.000000
NO_medical_doc                    5.316032
No_nurses                         5.316032
Institution_name                 21.808288
Hosp_inst_no                     21.703642
(%) of health exp against GDP     5.420678
Crude_death_rate                  0.104646
Expectancy_Age                    0.397656
Population                        0.125576
dtype: float64


In [24]:
# Define the subset of columns to consider
subset_columns = ['Crude_death_rate', 'Expectancy_Age', 'Population', '(%) of health exp against GDP','NO_medical_doc','No_nurses']

# Drop rows with null values in the  specified columns
infrastructure = infrastructure.dropna(subset=subset_columns, how='any')
checker(infrastructure)

Duplicated values: 0
*******
Null values:
 ParentLocation_x                 13.733333
Location                          0.000000
Period                            0.000000
NO_medical_doc                    0.000000
No_nurses                         0.000000
Institution_name                 22.622222
Hosp_inst_no                     22.511111
(%) of health exp against GDP     0.000000
Crude_death_rate                  0.000000
Expectancy_Age                    0.000000
Population                        0.000000
dtype: float64


In [25]:
# Define the subset of columns to consider
subset_columns = ['Institution_name', 'Hosp_inst_no']

# Drop rows with null values in all specified columns, not either or, but all two
infrastructure = infrastructure.dropna(subset=subset_columns, how='all')
checker(infrastructure)

Duplicated values: 0
*******
Null values:
 ParentLocation_x                 13.019788
Location                          0.000000
Period                            0.000000
NO_medical_doc                    0.000000
No_nurses                         0.000000
Institution_name                  0.143390
Hosp_inst_no                      0.000000
(%) of health exp against GDP     0.000000
Crude_death_rate                  0.000000
Expectancy_Age                    0.000000
Population                        0.000000
dtype: float64


In [26]:
# Define the subset of columns to consider
subset_columns = ['ParentLocation_x', 'Institution_name']

# Fill null values in the 'ParentLocation_x' and 'Institution_name' columns with the value "unspecified"
infrastructure[subset_columns] = infrastructure[subset_columns].fillna("unspecified")

# Call the 'checker' function to perform further checks or actions on the 'infrastructure' DataFrame
checker(infrastructure)


Duplicated values: 0
*******
Null values:
 ParentLocation_x                 0.0
Location                         0.0
Period                           0.0
NO_medical_doc                   0.0
No_nurses                        0.0
Institution_name                 0.0
Hosp_inst_no                     0.0
(%) of health exp against GDP    0.0
Crude_death_rate                 0.0
Expectancy_Age                   0.0
Population                       0.0
dtype: float64


In [27]:
infrastructure.shape

(3487, 11)

In [28]:
infrastructure

,ParentLocation_x,Location,Period,NO_medical_doc,No_nurses,Institution_name,Hosp_inst_no,(%) of health exp against GDP,Crude_death_rate,Expectancy_Age,Population
5,Americas,Colombia,2021,121659.000000,74626.000000,unspecified,2267.626583,7.484167,7.735,72.830000,51516562.0
6,Europe,Lithuania,2021,13795.000000,25736.000000,Publicly owned hospitals,67.000000,6.601667,17.000,74.339024,2800839.0
7,Europe,Lithuania,2021,13795.000000,25736.000000,General hospitals,23.920000,6.601667,17.000,74.339024,2800839.0
8,Europe,Lithuania,2021,13795.000000,25736.000000,Not-for-profit privately owned hospitals,0.000000,6.601667,17.000,74.339024,2800839.0
9,Europe,Lithuania,2021,13795.000000,25736.000000,For-profit privately owned hospitals,10.000000,6.601667,17.000,74.339024,2800839.0
...,...,...,...,...,...,...,...,...,...,...,...
7833,unspecified,Croatia,2021,13383.747573,27044.465753,Not-for-profit privately owned hospitals,0.250000,6.998142,16.200,76.424390,3879000.0
7834,unspecified,Croatia,2021,13383.747573,27044.465753,General hospitals,30.000000,6.998142,16.200,76.424390,3879000.0
7835,unspecified,Croatia,2021,13383.747573,27044.465753,For-profit privately owned hospitals,8.000000,6.998142,16.200,76.424390,3879000.0
7836,unspecified,Croatia,2021,13383.747573,27044.465753,Hospitals,17.220000,6.998142,16.200,76.424390,3879000.0
